In [1]:
import numpy as np
import scipy as sp
import numpy.random as random
import vegas

Particle class

In [2]:
class particle:
     def __init__(self, alive, mu, x, weight):
        self.alive = alive
        self.mu = mu
        self.x = x
        self.weight = weight

Monte Carlo function with inputs

$\sigma_s$, $\sigma_a$, $q_0$, $L$, $W$

In [25]:
def mc(sigmaS, sigmaA, q, L, x, mu):
    sigmaT = sigmaS + sigmaA
    tally = np.zeros(2)
        
    #create particle
    if mu < 0:
        mu = -1
    else:
        mu = 1
    p = particle(True, mu, x, q)

    while p.alive:
        #calculate distance to collision
        distance_to_collision = abs(-np.log(random.random())/sigmaT)

        #calculate distance to boundary
        if p.mu == -1:
            distance_to_boundary = p.x
        elif p.mu == 1:
            distance_to_boundary = L-p.x

        #tally if distance to collision is greater than distance to boundary
        if distance_to_collision > distance_to_boundary:
            if p.mu == -1:
                tally[0] += p.weight
                p.alive = False
            elif p.mu == 1:
                tally[1] += p.weight
                p.alive = False
        else:
            p.x += p.mu*distance_to_collision
            if random.random() < sigmaA/sigmaT: #absorbed
                p.alive = False
            elif random.random() < 0.5: #backscatter
                    p.mu *= -1

    #return weighted sum and tally
    return [sum(tally)*p.weight, tally[0]*p.weight, tally[1]*p.weight]

In [13]:
sigmaS = 1
sigmaA = .01

q = 1
L = 5
W = 2

mc(sigmaS, sigmaA, q, L, 1, 0)

[1.0, 1.0, 0.0]

### Vegas

In [26]:
sigmaS = 1
sigmaA = .1
q = 2
L = 5

def f(x):
    return mc(sigmaS, sigmaA, q, L, x[0], x[1])
    
integ = vegas.Integrator([[0, 2], [-1,1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('Total leakage = ', result[0])
print('Left leakage = ', result[1])
print('Right leakage = ', result[2])

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   11.02(23)       11.02(23)           0.00     1.00
  2   11.16(25)       11.09(17)           0.06     0.98
  3   11.19(24)       11.12(14)           0.12     0.99
  4   11.05(26)       11.10(12)           0.34     0.96
  5   11.14(25)       11.10(11)           0.53     0.89
  6   10.88(28)       11.06(10)           0.66     0.82
  7   10.92(25)       11.037(94)          0.72     0.79
  8   10.79(26)       11.009(88)          0.66     0.88
  9   10.95(28)       11.003(84)          0.61     0.93
 10   11.10(25)       11.014(80)          0.55     0.97

Total leakage =  11.014(80)
Left leakage =  8.606(81)
Right leakage =  2.408(58)
